## Amazon Athena SymlinkTextInputFormat Tables Over Apache Hudi Tables <a name="top"></a>

## Table of Contents:


1. [Overview](#Overview)
2. [Create or Update the Athena Table](#Create-or-Update-the-Athena-Table)
2. [Create or Update the symlink text files](#Create-or-Update-the-symlink-text-files)
3. [Load All the Partitions](#Load-All-the-Partitions)

<div class="alert alert-block alert-info">
<b>Note : </b>This notebook is tested on Apache Hudi 0.5.2-incubating on Amazon EMR 5.30</div>

### Overview
[(Back to the top)](#top)

Amazon Athena does not natively support Apache Hudi tables today. But as Apache Hudi tables are in Parquet, if we point Amazon Athena to the latest files in our Apache Hudi dataset, then Amazon Athena should be able to read them.

In this notebook, we demonstrate how to create SymlinkTextInputFormat tables to point Amazon Athena to the latest Apache Hudi files. This will allow Amazon Athena to query the the latest version of Parquet files in our Apache Hudi Dataset.

Note that the same Parquet table can be queried by Amazon Redshift Spectrum or any other engine that supports reading symlinkformat tables and Parquet files.

In [1]:
%%configure -f
{
    "conf":  { 
             "spark.jars":"hdfs:///httpclient-4.5.9.jar,hdfs:///hudi-spark-bundle.jar,hdfs:///spark-avro.jar,"
           } 
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
24,application_1591058751782_0044,spark,idle,Link,Link,


In [2]:
import org.apache.hudi.hive.HiveSyncConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor
import collection.JavaConverters._
import org.apache.hadoop.hive.conf.HiveConf
import org.apache.hudi.hive.HoodieHiveClient
import org.apache.hadoop.fs.Path
import org.apache.hudi.common.util.Option
import org.apache.hudi.hive.util.SchemaUtil
import org.apache.hudi.common.table.HoodieTableMetaClient
import org.apache.hudi.common.util.FSUtils
import org.apache.hudi.common.model.HoodieCommitMetadata
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.common.table.view.HoodieTableFileSystemView
import org.apache.hudi.table.HoodieTable
import org.apache.spark.SerializableWritable


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
26,application_1591058751782_0046,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.hudi.hive.HiveSyncConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor
import collection.JavaConverters._
import org.apache.hadoop.hive.conf.HiveConf
import org.apache.hudi.hive.HoodieHiveClient
import org.apache.hadoop.fs.Path
import org.apache.hudi.common.util.Option
import org.apache.hudi.hive.util.SchemaUtil
import org.apache.hudi.common.table.HoodieTableMetaClient
import org.apache.hudi.common.util.FSUtils
import org.apache.hudi.common.model.HoodieCommitMetadata
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.common.table.view.HoodieTableFileSystemView
import org.apache.hudi.table.HoodieTable
import org.apache.spark.SerializableWritable


In [3]:
val tablePath = "s3://neilawstmp2/tmp/hudi/example_hudi_partitioned_table"

val hiveTablePartitionFieldsList = "year,month"
val hiveSyncConfig = new HiveSyncConfig()
hiveSyncConfig.basePath = tablePath
hiveSyncConfig.jdbcUrl = "jdbc:hive2://localhost:10000"
hiveSyncConfig.databaseName = "default"
hiveSyncConfig.tableName = "example_hudi_partitioned_table"
val athena_tableName=hiveSyncConfig.tableName+"_athena"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tablePath: String = s3://neilawstmp2/tmp/hudi/example_hudi_partitioned_table
hiveTablePartitionFieldsList: String = year,month
hiveSyncConfig: org.apache.hudi.hive.HiveSyncConfig = HiveSyncConfig{databaseName='null', tableName='null', hiveUser='null', hivePass='null', jdbcUrl='null', basePath='null', partitionFields=[], partitionValueExtractorClass='org.apache.hudi.hive.SlashEncodedDayPartitionValueExtractor', assumeDatePartitioning=false, usePreApacheInputFormat=false, useJdbc=true, help=false}
hiveSyncConfig.basePath: String = s3://neilawstmp2/tmp/hudi/example_hudi_partitioned_table
hiveSyncConfig.jdbcUrl: String = jdbc:hive2://localhost:10000
hiveSyncConfig.databaseName: String = default
hiveSyncConfig.tableName: String = example_hudi_partitioned_table
athena_tableName: String = example_hudi_partitioned_table_athena


In [4]:
hiveSyncConfig.partitionFields =  hiveTablePartitionFieldsList.split(",").map(_.trim).filter(!_.isEmpty).toList.asJava
hiveSyncConfig.partitionValueExtractorClass = classOf[MultiPartKeysValueExtractor].getName

var hadoopConf = spark.sparkContext.hadoopConfiguration
val metaClient = new HoodieTableMetaClient(hadoopConf, tablePath, true)

val hiveConfig = new HiveConf()
hiveConfig.addResource(metaClient.getFs.getConf)

val hoodieHiveClient = new HoodieHiveClient(hiveSyncConfig, hiveConfig, metaClient.getFs)
val dataSchema = hoodieHiveClient.getDataSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hiveSyncConfig.partitionFields: java.util.List[String] = [year, month]
hiveSyncConfig.partitionValueExtractorClass: String = org.apache.hudi.hive.MultiPartKeysValueExtractor
hadoopConf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, __spark_hadoop_conf__.xml
metaClient: org.apache.hudi.common.table.HoodieTableMetaClient = HoodieTableMetaClient{basePath='s3://neilawstmp2/tmp/hudi/example_hudi_partitioned_table', metaPath='s3://neilawstmp2/tmp/hudi/example_hudi_partitioned_table/.hoodie', tableType=COPY_ON_WRITE}
hiveConfig: org.apache.hadoop.hive.conf.HiveConf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, emrfs-site.xml, org.apache.hadoop.hive.conf.LoopingByteArrayInputStream@7b8c6e50, file:/etc/spark/conf.dist/hive-site.xml
hoodieHiveClient

### Create or Update the Athena Table
[(Back to the top)](#top)

Let's create the Athena Parquet table or alter the table if the schema has changed. Our original table is called `example_hudi_partitioned_table` and we have decided to call the corresponding SymlinkTextInputFormat table `example_hudi_partitioned_table_athena`.

We will create a `.athena` folder within the S3 folder for the original table to create the Partitions and Symlink files.

In [5]:
if (!hoodieHiveClient.doesTableExist(athena_tableName))
{
  val serDeClassName = "org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe"
  val inputFormatClassName =  "org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat"
  val outputFormatClassName = "org.apache.hadoop.hive.ql.io.IgnoreKeyTextOutputFormat"

  val createTableDDL = SchemaUtil.generateCreateDDL(athena_tableName, dataSchema, hiveSyncConfig, inputFormatClassName,outputFormatClassName,serDeClassName)
  println(createTableDDL)  
  hoodieHiveClient.updateHiveSQL(createTableDDL)
  val alterTableDDL = new StringBuilder("ALTER TABLE ").append("`").append(hiveSyncConfig.databaseName)
    .append(".").append(athena_tableName).append("`").append(" SET LOCATION \"").append(hiveSyncConfig.basePath)
    .append("/.athena\"")
  println(alterTableDDL.toString())
  hoodieHiveClient.updateHiveSQL(alterTableDDL.toString())
} else{
  val tableSchema = hoodieHiveClient.getTableSchema(athena_tableName)
  val schemaDiff = SchemaUtil.getSchemaDifference(dataSchema, tableSchema, hiveSyncConfig.partitionFields)
  if (!schemaDiff.isEmpty)
  {
    val newSchemaStr = SchemaUtil.generateSchemaString(dataSchema, hiveSyncConfig.partitionFields)
    val alterTableDDL = new StringBuilder("ALTER TABLE ").append("`").append(hiveSyncConfig.databaseName)
      .append(".").append(athena_tableName).append("`").append(" REPLACE COLUMNS(").append(newSchemaStr)
      .append(" )")
    hoodieHiveClient.updateHiveSQL(alterTableDDL.toString())
  }
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

CREATE EXTERNAL TABLE  IF NOT EXISTS `default`.`example_hudi_partitioned_table_athena`( `_hoodie_commit_time` string, `_hoodie_commit_seqno` string, `_hoodie_record_key` string, `_hoodie_partition_path` string, `_hoodie_file_name` string, `id` bigint, `sk` bigint, `txt` string, `partitionKey` string) PARTITIONED BY (`year` string,`month` bigint) ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.IgnoreKeyTextOutputFormat' LOCATION 's3://neilawstmp2/tmp/hudi/example_hudi_partitioned_table'
ALTER TABLE `default.example_hudi_partitioned_table_athena` SET LOCATION "s3://neilawstmp2/tmp/hudi/example_hudi_partitioned_table/.athena"


### Create or Update the symlink text files
[(Back to the top)](#top)

Next we will create or update the symlink.txt files. We will first read the partitions and then drop the symlink.txt into the partition folders.

In [6]:
val activeTimeline = metaClient.getActiveTimeline
val lastCommit = activeTimeline.getCommitTimeline.lastInstant.get
val writerConfig = HoodieWriteConfig.newBuilder().withPath(tablePath).build()
val hoodieTable = HoodieTable.getHoodieTable(metaClient, writerConfig, spark.sparkContext)
val serializedConf = new SerializableWritable(hadoopConf)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

activeTimeline: org.apache.hudi.common.table.timeline.HoodieActiveTimeline = org.apache.hudi.common.table.timeline.HoodieActiveTimeline: [20200527222733__clean__COMPLETED],[20200527222733__commit__COMPLETED]
lastCommit: org.apache.hudi.common.table.timeline.HoodieInstant = [20200527222733__commit__COMPLETED]
writerConfig: org.apache.hudi.config.HoodieWriteConfig = org.apache.hudi.config.HoodieWriteConfig@75d184a8
hoodieTable: org.apache.hudi.table.HoodieTable[Nothing] = org.apache.hudi.table.HoodieCopyOnWriteTable@74e55f30
serializedConf: org.apache.spark.SerializableWritable[org.apache.hadoop.conf.Configuration] = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, emrfs-site.xml, __spark_hadoop_conf__.xml


In [7]:
val partitionPaths=FSUtils.getAllPartitionPaths(metaClient.getFs,tablePath,false)
val allPartitionPaths = sc.parallelize(partitionPaths.asScala)
allPartitionPaths.take(10).foreach(println)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

partitionPaths: java.util.List[String] = [year=2019/month=1, year=2019/month=10, year=2019/month=11, year=2019/month=12, year=2019/month=2, year=2019/month=3, year=2019/month=4, year=2019/month=5, year=2019/month=6, year=2019/month=7, year=2019/month=8, year=2019/month=9]
allPartitionPaths: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[0] at parallelize at <console>:44
year=2019/month=1
year=2019/month=10
year=2019/month=11
year=2019/month=12
year=2019/month=2
year=2019/month=3
year=2019/month=4
year=2019/month=5
year=2019/month=6
year=2019/month=7


Now that we have the partitions, lets create the 'symlink.txt' files into the .athena folder. We will parallelize the problem using Apache Spark by assigning the work to the executors.

In [8]:
hadoopConf=null
allPartitionPaths.map(partitionPath => {
  val view = hoodieTable.getBaseFileOnlyView
  val latestFiles = view.getLatestBaseFilesBeforeOrOn(partitionPath, lastCommit.toString).iterator.asScala.map( s => s.getPath).toList
  val dest = new Path(tablePath + "/.athena/" + partitionPath + "/symlink.txt")
  val fs1 = dest.getFileSystem(serializedConf.value)
  val out =  fs1.create(dest, true)
  out.write(latestFiles.mkString("\n").getBytes)
  out.close()
}).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hadoopConf: org.apache.hadoop.conf.Configuration = null
res8: Array[Unit] = Array((), (), (), (), (), (), (), (), (), (), (), ())


### Load All the Partitions
[(Back to the top)](#top)

Next we will load the partitions to the Metastore using MSCK REPAIR

In [9]:
val msckRepairTableDDL = new StringBuilder("MSCK REPAIR TABLE ").append("`").append(hiveSyncConfig.databaseName).append(".").append(athena_tableName).append("`")
hoodieHiveClient.updateHiveSQL(msckRepairTableDDL.toString())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

msckRepairTableDDL: StringBuilder = MSCK REPAIR TABLE `default.example_hudi_partitioned_table_athena`


Now you should be able to query the `example_hudi_partitioned_table_athena` in Amazon Athena.

If we run a ```SHOW CREATE TABLE `example_hudi_partitioned_table_athena````, we can see that it is using SymlinkTextInputFormat as the INPUTFORMAT and ParquetHiveSerDe as the SERDE.

```
CREATE EXTERNAL TABLE `example_hudi_partitioned_table_athena`(
  `_hoodie_commit_time` string, 
  `_hoodie_commit_seqno` string, 
  `_hoodie_record_key` string, 
  `_hoodie_partition_path` string, 
  `_hoodie_file_name` string, 
  `id` bigint, 
  `sk` bigint, 
  `txt` string, 
  `partitionkey` string)
PARTITIONED BY ( 
  `year` string, 
  `month` bigint)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://neilawstmp2/tmp/hudi/example_hudi_partitioned_table/.athena'
TBLPROPERTIES (
  'last_modified_by'='anonymous', 
  'last_modified_time'='1591648275')
```